# Game Benchmark Analysis
Este notebook carga un archivo JSON de resultados de `game_benchmark` y genera visualizaciones clave para el análisis de rendimiento de agentes.

## 1. Setup

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_theme(style="whitegrid")
%matplotlib inline

In [ ]:
# Cargar datos
FILE_PATH = "../tacticfps_test_fixed.json"  # Cambia esto por tu archivo JSON

with open(FILE_PATH, "r") as f:
    data = json.load(f)

matches = data["matches"]
print(f"Cargados {len(matches)} partidas de {matches[0]['game']}")

## 2. Leaderboard

In [ ]:
rows = []
for entry in data["leaderboard"]:
    name, stats = entry
    rows.append({
        "Agent": name,
        "ELO": stats["elo"],
        "Wins": stats["wins"],
        "Losses": stats["losses"],
        "Draws": stats["draws"]
    })

df_leaderboard = pd.DataFrame(rows).sort_values("ELO", ascending=False)
display(df_leaderboard)

plt.figure(figsize=(10, 5))
sns.barplot(data=df_leaderboard, x="ELO", y="Agent", palette="viridis")
plt.title("ELO Rating por Agente")
plt.show()

## 3. Matchup Matrix

In [ ]:
# Crear matriz de wins
agents = df_leaderboard["Agent"].unique()
matrix = pd.DataFrame(0, index=agents, columns=agents)

for m in matches:
    sides = m["sides"]
    winner = m["winner"]
    p1 = sides.get("player_1")
    p2 = sides.get("player_2")
    
    if not p1 or not p2: continue
    
    if winner == "player_1":
        matrix.loc[p1, p2] += 1
    elif winner == "player_2":
        matrix.loc[p2, p1] += 1
        
plt.figure(figsize=(8, 6))
sns.heatmap(matrix, annot=True, fmt="d", cmap="Blues")
plt.title("Win Matrix (Filas ganan a Columnas)")
plt.ylabel("Winner")
plt.xlabel("Loser")
plt.show()

## 4. Why Side Swap Matters
Esta sección analiza si existe una ventaja inherente a jugar como Player 1 (ej. Blue Team, Terrorist, etc).

In [ ]:
p1_wins = 0
p2_wins = 0
draws = 0

for m in matches:
    w = m.get("winner")
    if w == "player_1": p1_wins += 1
    elif w == "player_2": p2_wins += 1
    else: draws += 1

total = len(matches)
labels = ["Player 1 Wins", "Player 2 Wins", "Draws"]
sizes = [p1_wins, p2_wins, draws]

plt.figure(figsize=(6, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=["#3498db", "#e74c3c", "#bdc3c7"])
plt.title(f"Side Winrate Distribution (Total: {total})")
plt.show()

In [ ]:
# Desglose por Agente: ¿Son consistentes cambiando de lado?
agent_side_stats = []

for agent in agents:
    as_p1 = [m for m in matches if m["sides"].get("player_1") == agent]
    as_p2 = [m for m in matches if m["sides"].get("player_2") == agent]
    
    wins_p1 = sum(1 for m in as_p1 if m["winner"] == "player_1")
    wins_p2 = sum(1 for m in as_p2 if m["winner"] == "player_2")
    
    wr_p1 = (wins_p1 / len(as_p1) * 100) if as_p1 else 0
    wr_p2 = (wins_p2 / len(as_p2) * 100) if as_p2 else 0
    
    agent_side_stats.append({"Agent": agent, "Side": "Player 1", "Winrate": wr_p1})
    agent_side_stats.append({"Agent": agent, "Side": "Player 2", "Winrate": wr_p2})

df_side = pd.DataFrame(agent_side_stats)

plt.figure(figsize=(10, 5))
sns.barplot(data=df_side, x="Agent", y="Winrate", hue="Side", palette="muted")
plt.title("Winrate por Agente y Lado")
plt.ylim(0, 100)
plt.show()

## 5. Score Difference Distribution
Margen de victoria.

In [ ]:
diffs = []
for m in matches:
    scores = m.get("final_scores", {})
    s1 = scores.get("player_1", 0)
    s2 = scores.get("player_2", 0)
    diffs.append(s1 - s2)

plt.figure(figsize=(10, 5))
sns.histplot(diffs, kde=True, bins=20)
plt.title("Score Difference (P1 - P2)")
plt.xlabel("Difference (>0 favors P1, <0 favors P2)")
plt.show()